In [ ]:
import yaml
import numpy as np
from utils import *
import networkx as nx
import matplotlib.pyplot as plt
from collections import defaultdict
from itertools import combinations

In [ ]:
path = "../data/bio/bio-nr-105-size-20.graph"
n = 20

In [ ]:
out = !../cmake-build-release/fpt --input {path} --search-strategy Fixed --k 20000 --all 1 --F C4P4

In [ ]:
doc = list(yaml.safe_load_all("\n".join(out)))[0]

In [ ]:
doc["solutions"][-1]["cost"]

In [ ]:
c = np.zeros((n, n))
C = len(doc["solutions"])

for solution in doc["solutions"]:
    for (u, v) in solution["edits"]:
        c[u, v] += 1
        c[v, u] += 1

p = c / C

In [ ]:
G, S = read_metis_graph(path)

A = ((1 + np.sign(S)) / 2)
A[A == 1] = 1 - p[A == 1]
A[A == 0] = p[A == 0]

In [ ]:
for q in np.linspace(0.1, 0.9, 9):
    G = nx.from_numpy_matrix(A > q)
    w = 5 * np.array([A[u, v] - q for u, v in G.edges()])
    pos = nx.spring_layout(G)
    nx.draw_networkx_nodes(G, pos)
    nx.draw_networkx_edges(G, pos, width=w)
    plt.show()

In [ ]:
clusterings = defaultdict(list)

for solution in doc["solutions"]:
    E = nx.Graph()
    E.add_nodes_from(range(n))
    E.add_edges_from(set(G.edges()) ^ set(map(tuple, solution["edits"])))
    clustering = tuple(tuple(S) for S in nx.connected_components(E))
    clusterings[clustering] += [solution["cost"]]

In [ ]:
clusterings

In [ ]:
m = 100
k_max = 20000

fig, ax = plt.subplots(figsize=(8, 4))
ax.set_yscale("log")

for c, ks in clusterings.items():
    if ks[0] <= k_max and len(ks) > 5:
        ax.scatter(np.array(ks) / m, range(len(ks)), label=f"{len(ks)} {c}")

ax.set_xlim((0, k_max / m))
ax.set_ylim((1, 1000))
# fig.legend(loc="upper left", bbox_to_anchor=(1, 1))
fig.tight_layout()
plt.show()